In [1]:
# !pip install nltk pandas numpy scipy keras tensorflow sklearn
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
import json
import nltk
import lda2
import string
import copy
import pandas as pd
import numpy as np
import keras.backend as K

from keras import regularizers
from keras.models import Model
from numpy import linalg as LA
from nltk.corpus import stopwords
from scipy.special import gammaln
from keras.models import Sequential
from scipy.sparse import csr_matrix
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Activation, Embedding, LSTM

In [3]:
def preprocess(pd):
    pd = pd.str.lower()
    pd = pd.str.replace('[{}]'.format(string.punctuation), ' ')
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])
    pd = pd.apply(lambda x: [item for item in x if item not in stop_words])
    return pd.str.join(' ')

In [4]:
def get_x_lstm(max_vocab, vocab):
    tokenizer = Tokenizer(nb_words=max_vocab, lower=True, split=' ')
    tokenizer.fit_on_texts(vocab)
    X = tokenizer.texts_to_sequences(vocab)
    return pad_sequences(X)

In [5]:
def sampleFromDirichlet(alpha):
    return np.random.dirichlet(alpha)

def sampleFromCategorical(theta):
#     theta = theta/np.sum(theta)
    theta = np.exp(theta)/np.exp(theta).sum()
    return np.random.multinomial(1, theta).argmax()

def word_indices(wordOccuranceVec):
    for idx in wordOccuranceVec.nonzero()[0]:
        for i in range(int(wordOccuranceVec[idx])):
            yield idx

In [6]:
# class SentimentLDAGibbsSampler:
#     def __init__(self, numTopics, alpha, beta):
#         self.alpha = alpha
#         self.beta = beta
#         self.numTopics = numTopics
#         self.probabilities_t = {}

#     def processReviews(self, reviews):
#         self.vectorizer = CountVectorizer(analyzer="word",tokenizer=None,preprocessor=None,
#                                           stop_words="english",max_features=MAX_VOCAB_SIZE,max_df=.5,min_df=5)
#         train_data_features = self.vectorizer.fit_transform(reviews)
#         words = self.vectorizer.get_feature_names()
#         self.vocabulary = dict(zip(words,np.arange(len(words))))
#         self.inv_vocabulary = dict(zip(np.arange(len(words)),words))
#         wordOccurenceMatrix = train_data_features.toarray()
#         return wordOccurenceMatrix

#     def _initialize_(self, reviews, V_weights):
#         self.wordOccuranceMatrix = self.processReviews(reviews)
#         numDocs, vocabSize = self.wordOccuranceMatrix.shape
        
#         # Pseudocounts
#         self.n_dt = np.zeros((numDocs, self.numTopics))
#         self.n_t = np.zeros((self.numTopics))
#         self.n_d = np.zeros((numDocs))
#         self.n_vt = np.zeros((vocabSize, self.numTopics))
#         self.dt_distribution = np.zeros((numDocs, self.numTopics))
# #         self.t_distribution = np.zeros((self.numTopics))
#         self.topics = {}

#         self.alphaVec = self.alpha.copy()

#         for d in range(numDocs):            
# #             topicDistribution = sampleFromDirichlet(self.alphaVec)
#             topicDistribution = V_weights[d]
            
#             for i, w in enumerate(word_indices(self.wordOccuranceMatrix[d, :])):
#                 t = sampleFromCategorical(topicDistribution)

#                 self.topics[(d, i)] = t
#                 self.n_dt[d, t] += 1
#                 self.n_d[d] += 1
#                 self.n_vt[w, t] += 1
#                 self.n_t[t] += 1

#             self.dt_distribution[d,:] = (self.n_dt[d] + self.alphaVec) / (self.n_d[d] + np.sum(self.alphaVec))
            
#     def conditionalDistribution(self, d, v):
#         probabilities_t = np.ones((self.numTopics))
        
#         firstFactor = (self.n_dt[d,:] + self.alphaVec) / (self.n_d[d] + np.sum(self.alphaVec))
#         thirdFactor = (self.n_vt[v, :] + self.beta) / (self.n_t + self.n_vt.shape[0] * self.beta)

#         probabilities_t *= firstFactor[:]
#         probabilities_t *= thirdFactor
#         probabilities_t /= np.sum(probabilities_t)
#         return probabilities_t

#     def run(self, reviews, V_weights, maxIters=100):
#         self._initialize_(reviews, V_weights)
#         self.loglikelihoods = np.zeros(maxIters)
#         numDocs, vocabSize = self.wordOccuranceMatrix.shape

#         for iteration in range(maxIters):
#             print ("Starting iteration %d of %d" % (iteration + 1, maxIters))
#             loglikelihood = 0
#             for d in range(numDocs):
#                 for i, v in enumerate(word_indices(self.wordOccuranceMatrix[d, :])):
#                     t = self.topics[(d, i)]
                
#                     self.n_dt[d, t] -= 1
#                     self.n_d[d] -= 1
#                     self.n_t[t] -= 1
#                     self.n_vt[v, t] -= 1

#                     probabilities_t = self.conditionalDistribution(d, v)
#                     t = sampleFromCategorical(probabilities_t)
#                     self.probabilities_t[(d,v)] = probabilities_t[t]
                    
#                     self.topics[(d, i)] = t
#                     self.n_dt[d, t] += 1
#                     self.n_d[d] += 1
#                     self.n_vt[v, t] += 1
#                     self.n_t[t] += 1
                
#                 if iteration == maxIters - 1:
#                     self.dt_distribution[d,:] = (self.n_dt[d] + self.alphaVec) / (self.n_d[d] + np.sum(self.alphaVec))
#                     self.dt_distribution = self.dt_distribution/np.sum(self.dt_distribution, axis=1)[:,np.newaxis]

#                 loglikelihood += np.sum(gammaln((self.n_dt[d] + self.alphaVec)
#                                                )) - gammaln(np.sum((self.n_dt[d] + self.alphaVec))) 
#                 loglikelihood -= np.sum(gammaln(self.alphaVec)) - gammaln(np.sum(self.alphaVec))

#             for k in range(self.numTopics):
#                 loglikelihood += (np.sum(gammaln((self.n_vt[:, k] + self.beta)
#                                                 )) - gammaln(np.sum((self.n_vt[:, k] + self.beta))))
#                 loglikelihood -= (vocabSize * gammaln(self.beta) - gammaln(vocabSize * self.beta))

#             self.loglikelihoods[iteration] = loglikelihood        
# #             print ("Total loglikelihood is {}".format(loglikelihood))

In [8]:
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop_words = stopwords.words('english')

In [9]:
dataset = []
with open("Automotive_5.json", 'r') as f:
    p = f.readlines()
    for i in p:
        dataset.append(json.loads(i))
dataset = pd.DataFrame(dataset)

In [10]:
U_user_ids = list(dataset.reviewerID.unique())
V_item_ids = list(dataset.asin.unique())

In [11]:
M_users_count = len(U_user_ids)
N_items_count = len(V_item_ids)

In [12]:
M_users_count, N_items_count

(2928, 1835)

In [13]:
numtopics = 5
MAX_VOCAB_SIZE = 100
lambda_u = lambda_v = 0.1
lambda_param = 0.8

In [14]:
R_rating = np.zeros((M_users_count, N_items_count))

for idx, i in enumerate(dataset[["reviewerID", "overall", "asin"]].values):
    R_rating[U_user_ids.index(i[0])][V_item_ids.index(i[2])] = i[1]
        
R_rating = R_rating/5

In [15]:
D_combined_reviews = [""] * N_items_count

for i in dataset[["asin", "reviewText"]].values:
    D_combined_reviews[V_item_ids.index(i[0])] += i[1]
D_combined_reviews = pd.DataFrame(D_combined_reviews)
D_combined_reviews = preprocess(D_combined_reviews[0])
D_combined_reviews.shape

(1835,)

In [16]:
def processReviews(reviews, window=5, MAX_VOCAB_SIZE=MAX_VOCAB_SIZE):
    vectorizer = CountVectorizer(analyzer="word",tokenizer=None)
    train_data_features = vectorizer.fit_transform(reviews)
    words = vectorizer.get_feature_names()
    vocabulary = dict(zip(words,np.arange(len(words))))
    inv_vocabulary = dict(zip(np.arange(len(words)),words))
    wordOccurenceMatrix = train_data_features.toarray()
    return wordOccurenceMatrix, vocabulary, words

In [17]:
matrix, vocabulary, words = processReviews(D_combined_reviews)

## PMF

In [18]:
U_user_weights = np.random.rand(numtopics, M_users_count)
V_item_weights = np.random.rand(numtopics, N_items_count)

In [19]:
U_user_weights.shape, V_item_weights.shape

((5, 2928), (5, 1835))

## HFT

In [20]:
beta = 0.01
alpha = 10/numtopics * np.ones(numtopics)

In [21]:
# sampler = SentimentLDAGibbsSampler(numtopics, alpha, beta)
sampler = lda2.LdaSampler(n_topics=numtopics, matrix_shape=matrix.shape, lambda_param=lambda_param)

In [22]:
numtopics, alpha, beta

(5, array([2., 2., 2., 2., 2.]), 0.01)

## LSTM

In [23]:
lstm_out = 128
batch_size = 8
p_embedding_lstm = 200

In [24]:
X = get_x_lstm(MAX_VOCAB_SIZE, D_combined_reviews.values)

/root/miniconda3/envs/python2/lib/python2.7/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [25]:
model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, p_embedding_lstm, input_length=X.shape[1]))
model.add(LSTM(lstm_out, dropout = 0.2))
model.add(Dense(5, activation='tanh', name ="doc_latent_vector", kernel_regularizer=regularizers.l2()))
model.compile(loss = 'mean_squared_error', optimizer='rmsprop', metrics = ['accuracy'])

W0628 12:53:44.348578 140557525628288 deprecation_wrapper.py:119] From /root/miniconda3/envs/python2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0628 12:53:44.373152 140557525628288 deprecation_wrapper.py:119] From /root/miniconda3/envs/python2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 12:53:44.377270 140557525628288 deprecation_wrapper.py:119] From /root/miniconda3/envs/python2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0628 12:53:44.542172 140557525628288 deprecation_wrapper.py:119] From /root/miniconda3/envs/python2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Pleas

In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2103, 200)         20000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               168448    
_________________________________________________________________
doc_latent_vector (Dense)    (None, 5)                 645       
Total params: 189,093
Trainable params: 189,093
Non-trainable params: 0
_________________________________________________________________


In [27]:
def get_last_layer_op():
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer('doc_latent_vector').output)
    return intermediate_layer_model.predict(X)

### Loss

In [28]:
def get_l1():
    l1 = 0
    for i in range(M_users_count):
        for j in range(N_items_count):
            if R_rating[i][j]>0:
                l1 += (R_rating[i][j] - np.dot(U_user_weights.T[i], V_item_weights.T[j]))**2
    return l1

In [29]:
def get_l3():
    return LA.norm(U_user_weights, 'fro')

In [30]:
def get_l4():
    return LA.norm(V_item_weights.T - get_last_layer_op(), 'fro')

In [31]:
def get_total_loss():
    return get_l1() + lambda_u * get_l3() + lambda_v * get_l4()

## Gradient V

In [32]:
def get_gradient_V(sampler, lstm_last_layer):
    param_k = 0.1
    peakiness = 1
    lambda_t = 0.01
    param_Nj = matrix.sum(axis=1)
    param_njk = sampler.nmz.copy()
    dt_distribution = sampler.theta()

    diff_lv = []
    for j in range(N_items_count):
        temp_sums = [0]*5
        for i in range(M_users_count):
            if R_rating[i][j]>0:
                temp_sums += (R_rating[i][j] - np.dot(U_user_weights.T[i], V_item_weights.T[j]))*U_user_weights.T[i]
        temp_sums += 2 * lambda_v * (V_item_weights.T[j] - lstm_last_layer[j])
        temp_sums -= lambda_t*peakiness*(param_njk[j] - param_Nj[j]*dt_distribution[j]).sum()
        diff_lv.append(list(temp_sums))
    diff_lv = np.array(diff_lv)
    # diff_lv -= lambda_t*peakiness*(param_njk - param_Nj*sampler.dt_distribution)
    return diff_lv

## Gradinet U

In [33]:
def get_gradient_U():
    diff_lu = []

    for i in range(M_users_count):
        temp_sums = [0]*5
        for j in range(N_items_count):
            if R_rating[i][j]>0:
                temp_sums += (R_rating[i][j] - np.dot(U_user_weights.T[i], V_item_weights.T[j]))*V_item_weights.T[j]
        temp_sums += 2 * lambda_u * U_user_weights.T[i]
        diff_lu.append(list(temp_sums))
    diff_lu = np.array(diff_lu)
    return diff_lu

## Gradinet phi

In [34]:
def get_gradient_Phi(sampler, Phi_weights):
    param_nkw = sampler.nzw.T
    param_Nk = sampler.nzw.sum(axis=1)
    diff_phi = []
    for w in range(MAX_VOCAB_SIZE):
        param_zw = np.exp(Phi_weights[w]).sum()
        temp_phi = []
        for k in range(numtopics):
            temp_phi.append(param_nkw[w, k] - (param_Nk[k] * np.exp(Phi_weights[w, k])/param_zw))
        diff_phi.append(temp_phi)
    diff_phi = np.array(diff_phi)
    return diff_phi

# RUN

In [35]:
maxiter_hft = 10
learning_rate_pmf = learning_rate_hft = 0.01
Phi_weights = np.random.rand(MAX_VOCAB_SIZE, numtopics)

In [36]:
iterations = 10
for i in range(iterations):
    print("="*30, "\niteration", i)
    sampler.run(matrix, maxiter_hft)
    
    for i in range(5):
        model.fit(X, V_item_weights.T, epochs=5, batch_size=128)
        lstm_last_layer = get_last_layer_op()

        print("\nExtracting Gradients...")
        gradient_v = get_gradient_V(sampler, lstm_last_layer)
        gradient_u = get_gradient_U()
        gradient_phi = get_gradient_Phi(sampler, Phi_weights)

        print("\nUpdating Gradients...")
        U_user_weights -= learning_rate_pmf * gradient_u.T
        V_item_weights -= learning_rate_pmf * gradient_v.T
        Phi_weights -= learning_rate_hft * gradient_phi
    
        print(get_l1(), get_l3(), get_l4())

('==============================', '\niteration', 0)


W0628 12:53:45.325187 140557525628288 deprecation.py:323] From /root/miniconda3/envs/python2/lib/python2.7/site-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0628 12:53:46.582144 140557525628288 deprecation_wrapper.py:119] From /root/miniconda3/envs/python2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
1835/1835 [==============================] - 93s 51ms/step - loss: 0.2254 - acc: 0.1973
Epoch 2/5
1835/1835 [==============================] - 87s 48ms/step - loss: 0.1587 - acc: 0.1978
Epoch 3/5
1835/1835 [==============================] - 89s 48ms/step - loss: 0.1439 - acc: 0.2251
Epoch 4/5
1835/1835 [==============================] - 83s 45ms/step - loss: 0.1336 - acc: 0.2300
Epoch 5/5
1835/1835 [==============================] - 88s 48ms/step - loss: 0.1247 - acc: 0.2365

Extracting Gradients...

Updating Gradients...
(10023.844529642285, 71.85223904829498, 28.831003643404127)
Epoch 1/5
1835/1835 [==============================] - 86s 47ms/step - loss: 0.1236 - acc: 0.2458
Epoch 2/5
1835/1835 [==============================] - 86s 47ms/step - loss: 0.1176 - acc: 0.2403
Epoch 3/5
1835/1835 [==============================] - 88s 48ms/step - loss: 0.1117 - acc: 0.2436
Epoch 4/5
1835/1835 [==============================] - 85s 46ms/step - loss: 0.1073 - acc: 0.2578
Epoch 5/5


KeyboardInterrupt: 